In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

engine = create_engine("sqlite:///./app/connection/test.db", echo=False)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
session = SessionLocal()

In [ ]:
from app.services.kor_value_service import KorValueService


kor_value_service = KorValueService(session)
value_list = kor_value_service.get_values(isDf=True)

In [ ]:
# 밸류 지표 히스토그램
value_pivot = value_list.pivot(index="itemCd", columns="metrics", values="amt")
value_rank = value_pivot.rank(axis=0)

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(5, 1, figsize=(10, 6), sharex=True)
for n, ax in enumerate(axes.flatten()):
    ax.hist(value_rank.iloc[:, n])
    ax.set_title(value_rank.columns[n], size=12)

In [ ]:
value_pivot.isna().sum()

In [ ]:
# 랭킹을 구한 후 z-scoe로 정규화

from scipy.stats import zscore

value_rank_z = value_rank.apply(zscore, nan_policy="omit")

In [ ]:
fig, axes = plt.subplots(5, 1, figsize=(10, 6), sharex=True, sharey=True)
for n, ax in enumerate(axes.flatten()):
    ax.hist(value_rank_z.iloc[:, n])
    ax.set_title(value_rank.columns[n], size=12)

In [ ]:
# 멀티팩터
from app.services.portfolio_services.multi_factor_portfolio_service import (
    MultiFactorPortfolioService,
    OutlierMethod,
)


multi_factor_portfolio_service = MultiFactorPortfolioService(session)
final_portifolio = multi_factor_portfolio_service.get_final_portfolio(
    method=OutlierMethod.TRIM
)

In [ ]:
data_bind_final = multi_factor_portfolio_service._data_bind_final
data_bind = multi_factor_portfolio_service._data_bind

In [ ]:
final_portifolio[final_portifolio["invest"] == "Y"]

In [ ]:
# 그래프
multi_factor_portfolio_service.paint_graph(
    data_bind[["z_quality", "z_value", "z_momentum"]]
)

In [ ]:
# 그래프
multi_factor_portfolio_service.paint_graph(data_bind_final)